In [ ]:
# compare result from TOMOATT and fortran code

fpath_kernel_fort = './fortran/ega5/output/kernel_step1_sum'
fpath_synth_fort  = './fortran/ega5/output/syn_step1_event1' # Table
fpath_adj_fort    = './fortran/ega5/output/adj_step1_event1' # TableADJ
fpath_out_tomoatt = './out_data_sim_0.h5'

# grid information in fortran code
nr = 55
nt = 55
np = 55
npoints = nr*nt*np

# division
ndiv_r = 1 
ndiv_t = 1  
ndiv_p = 1   

In [ ]:
import h5py
import numpy


# read fortran test file and convert to numpy array
def read_fortran_result_file(fpath):
    with open(fpath, 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
        lines = [line for line in lines if line]
        lines = [line.split() for line in lines]
        lines = numpy.array(lines, dtype=numpy.float64)
        return lines

fortran_data = read_fortran_result_file(fpath_kernel_fort)
## change dimension of fortran data
Ks_fort = fortran_data[:,3].reshape(nr, nt, np)
Kxi_fort = fortran_data[:,4].reshape(nr, nt, np)
Keta_fort = fortran_data[:,5].reshape(nr, nt, np)

fortran_synth_data = read_fortran_result_file(fpath_synth_fort).reshape(nr, nt, np)
fortran_adj_data = read_fortran_result_file(fpath_adj_fort).reshape(nr, nt, np)

In [ ]:
# read h5 data
import sys
sys.path.append("../../utils/")

from tomoatt_data_retrieval import get_data_from_h5

#Ks_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/Ks_update_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p, verbose=True)
#Kxi_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/Kxi_update_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p)
#Keta_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/Keta_update_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p)

Ks_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/Ks_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p, verbose=True)
Kxi_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/Kxi_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p)
Keta_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/Keta_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p)

Syn_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/T_res_src_0_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p)
Adj_tomoatt = get_data_from_h5(fpath_out_tomoatt, "Data/adjoint_field_src_0_inv_0000", nr, nt, np, ndiv_r, ndiv_t, ndiv_p)

In [ ]:
# plot slice of Ks_inv
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def plot_slice(data_es, data_fort, slice_ax, id_slice, contour=False, make_anime=False, tag=""):

    if (slice_ax == 'r'):
        data_es_v   = data_es[  id_slice, :, :]
        data_fort_v = data_fort[id_slice, :, :]
    elif (slice_ax == 't'):
        data_es_v   = data_es[  :, id_slice, :]
        data_fort_v = data_fort[:, id_slice, :]
    elif (slice_ax == 'p'):
        data_es_v   = data_es[  :, :, id_slice]
        data_fort_v = data_fort[:, :, id_slice]
    else:
        print("Error: slice_ax should be 'r', 't' or 'p'")
        return

    # use the same color range for both tomoatt and fortran   
    combined_data = numpy.array([data_es_v.flatten(),data_fort_v.flatten()])
    #Get the min and max of all your data
    _min, _max = numpy.amin(combined_data), numpy.amax(combined_data)
    # make the color range symmetric
    if (abs(_min) > abs(_max)):
        if(_min < 0):
            _max = -_min
    else:
            _min = -_max



    # make the color range symmetric
    diff_data = data_es_v-data_fort_v
    _min_diff, _max_diff = numpy.amin(diff_data), numpy.amax(diff_data)
    if (abs(_min_diff) > abs(_max_diff)):
        if (_min_diff < 0):
            _max_diff = -_min_diff
    else:
        _min_diff = -_max_diff

    #cmap1="viridis"
    cmap1="seismic"
    cmap2="seismic"

    plt.figure(figsize=(20,10))

    plt.subplot(1,3,1)
    plt.imshow(data_es_v, cmap=cmap1, vmin=_min, vmax=_max)
    plt.colorbar()
    if(contour):
        plt.contour(data_es_v, colors='k', linewidths=0.5)
    plt.title('result_tomoatt')

    plt.subplot(1,3,2)
    plt.imshow(data_fort_v, cmap=cmap1, vmin=_min, vmax=_max)
    plt.colorbar()
    if(contour): 
        plt.contour(data_fort_v, colors='k', linewidths=0.5)
    plt.title('result_fort')
    
    plt.subplot(1,3,3)
    plt.imshow(diff_data, cmap=cmap2, vmin=_min_diff, vmax=_max_diff)
    plt.colorbar()
    if(contour):
        plt.contour(diff_data, colors='k', linewidths=0.5) 
    plt.title('tomoatt-fort')

    if (not make_anime):
        plt.show()
    else:
        plt.savefig("out_"+tag+"_"+slice_ax+str(id_slice).zfill(3)+".png")


In [ ]:
plot_slice(Syn_tomoatt, fortran_synth_data, 'r', 20, contour=True)

In [ ]:

def anime_slices(data_es, data_fort, slice_ax, tag):

    if (slice_ax == 'r'):
        n = nr
    elif (slice_ax == 't'):
        n = nt
    elif (slice_ax == 'p'):
        n = np

    for i in range(0, n):
        plot_slice(data_es, data_fort, slice_ax, i, contour=True, make_anime=True, tag=tag)

    ## make png files to gif
    import subprocess
    subprocess.call(["convert", "-delay", "20", "-loop", "0", "out_{}_{}*.png".format(tag,slice_ax), "out_{}_{}.gif".format(tag,slice_ax)])
    # erase png files
    for i in range(0, nr):
        subprocess.call(["rm", "out_{}_{}".format(tag,slice_ax)+str(i).zfill(3)+".png"])

In [ ]:
anime_slices(Ks_tomoatt, Ks_fort, 'r', 'Ks')
anime_slices(Ks_tomoatt, Ks_fort, 't', 'Ks')
anime_slices(Ks_tomoatt, Ks_fort, 'p', 'Ks')


In [ ]:
anime_slices(Adj_tomoatt, fortran_adj_data,'r', 'Adj')

In [ ]:
anime_slices(Keta_tomoatt, Keta_fort, 'r', 'Keta')
anime_slices(Keta_tomoatt, Keta_fort, 't', 'Keta')
anime_slices(Keta_tomoatt, Keta_fort, 'p', 'Keta')


In [ ]:
anime_slices(Kxi_tomoatt, Kxi_fort, 'r', 'Kxi')
anime_slices(Kxi_tomoatt, Kxi_fort, 't', 'Kxi')
anime_slices(Kxi_tomoatt, Kxi_fort, 'p', 'Kxi')


In [ ]:
anime_slices(Adj_tomoatt, fortran_adj_data, 'r', 'adj')
